In [1]:
import altair as alt
import pandas as pd
from vega_datasets import data
from IPython.display import display, HTML

display(HTML("""
<style>
form.vega-bindings {
    position: absolute;
    left: 100px;
    top: 600px;
}
</style>
"""))

states = alt.topo_feature(data.us_10m.url, 'states')
living_wage_df =  pd.read_csv('all_living_wage.csv')
living_wage_df.rename(columns={'Work1_Children_0': '1 Working Adult, 0 Child','Work1_Children_1': '1 Working Adult, 1 Child',
                              'Work1_Children_2': '1 Working Adult, 2 Childs','Work2_Children_0': '2 Working Adults, 0 Child',
                                'Work2_Children_1': '2 Working Adults, 1 Child','Work2_Children_2': '2 Working Adults, 2 Childs',
                              'Work1Adult1_Children_0':'1 Working Adult, 1 Nonworking Adult, 0 Child', 
                               'Work1Adult1_Children_1':'1 Working Adult, 1 Nonworking Adult, 1 Child',
                              'Work1Adult1_Children_2':'1 Working Adult, 1 Nonworking Adult, 2 Child'}, inplace=True)

min_wage_df =  pd.read_csv('Effective_Rate_2022_Dollars.csv')
min_wage_2022 = min_wage_df.query('Year == 2022')[['State', 'id', 'Effective_Minimum_Wage_2022_Dollars']]

living_wage = pd.melt(living_wage_df, ['State', 'id'])
living_wage.rename(columns={'value': 'Wage Rate $'}, inplace=True)

living_vs_min_wage = pd.merge(living_wage, min_wage_2022)
living_vs_min_wage.rename(columns={'Effective_Minimum_Wage_2022_Dollars': 'Min Wage $'}, inplace=True)

living_vs_min_wage['Min:Living_Ratio']=round(living_vs_min_wage['Min Wage $']/living_vs_min_wage['Wage Rate $'],3)

dropdown = alt.binding_select(options=list(living_vs_min_wage['variable'].unique()))
selection = alt.selection_single(name="Family Size",fields=['variable'], bind=dropdown)
click = alt.selection_multi(fields=['State'])

 
choropleth3 = (alt.Chart(living_vs_min_wage, title="2022 Minimum versus Living Wage").mark_geoshape(
    #   state borders are stroke and strokeWidth
    stroke='black',
    strokeWidth=1
    ).encode(
    #    more color schemes can be found at https://vega.github.io/vega/docs/schemes/
    color=alt.Color('Min:Living_Ratio', scale=alt.Scale(scheme='reds'), sort='descending'),
    opacity=alt.condition(click, alt.value(1), alt.value(0.2)),
    tooltip=[alt.Tooltip('State'),alt.Tooltip('Min Wage $', format='$'), 
             alt.Tooltip('Wage Rate $', format='$'), alt.Tooltip('Min:Living_Ratio')]
    ).add_selection(
        selection
    ).transform_filter(
        selection
    ).transform_lookup(
        lookup='id',
        from_=alt.LookupData(states, key='id', fields=['type', 'properties', 'geometry'])
).project('albersUsa'
).properties(
        width=800,
        height=600
)
)

top_10 = (alt.Chart(living_vs_min_wage, title="Top 10 Wage Ratios").mark_bar().encode(
    x='Min:Living_Ratio:Q',
    y=alt.Y('State:N', sort='-x'),
    color=alt.Color('Min:Living_Ratio:Q',legend=None, scale=alt.Scale(scheme='reds'), sort='descending'),
    opacity=alt.condition(click, alt.value(1), alt.value(0.2))
).add_selection(
    selection,
    click
).transform_filter(
    selection
).transform_window(
    rank='rank(Min:Living_Ratio)',
    sort=[alt.SortField('Min:Living_Ratio',order='descending')]
).transform_filter(
    (alt.datum.rank<=10)
))

bottom_10 = (alt.Chart(living_vs_min_wage, title="Bottom 10 Wage Ratios").mark_bar().encode(
    x='Min:Living_Ratio:Q',
    y=alt.Y('State:N', sort='x'),
    color=alt.Color('Min:Living_Ratio:Q',legend=None, scale=alt.Scale(scheme='reds')),
    opacity=alt.condition(click, alt.value(1), alt.value(0.2))
).add_selection(
    selection,
    click
).transform_filter(
    selection
).transform_window(
    rank='rank(Min:Living_Ratio)',
    sort=[alt.SortField('Min:Living_Ratio',order='ascending')]
).transform_filter(
    (alt.datum.rank<=10)
))

bar = top_10 | bottom_10

Min_vs_Live = (choropleth3 & bar)

Min_vs_Live.save('Min_vs_Living_Wage.html')

Min_vs_Live

alt.VConcatChart(...)